#### Basics ####

In [ ]:
### Getting Help

    Get-Command | where CommandType -eq 'Cmdlet' | Format-Wide -Column 2
    Get-Command -Noun Process | Format-Wide -Column 2

    Get-Process -?
    Get-Help Get-Process -Online
    Get-Help Get-Process -ShowWindow
    Get-Help Get-Process -Full

    Get-Help Get-Process -Parameter *
    Get-Help about_CommonParameters
    
#Powershell version
    $PSVersionTable

In [ ]:
### Output - Format

    Format-Table
    Format-Wide
    Format-List
    Format-Custom

    Get-Service | Sort-Object Name | Format-List Name, Status, CanStop
    Get-Process code | Format-Table Id, ProcessName

In [ ]:
### Output - Out

    Out-Default
    Out-File
    Out-GridView
    Out-Host
    Out-Null
    Out-String

    Get-Process code | Out-GridView -Title "Processes" -PassThru 

In [ ]:
In Powershell - a 300 ms (millisecond) delay is implemented before something is displayed in the console
As a result, the output will not be displayed in the order the commands imply
To prevent this proble, Out-Host might be added to the end of the pipeline to immediately display values in the console

In [ ]:
# Tee-Object

# The Tee-Object command is used to send output to two places at the same time. Tee-Object is used
# to write output to a console and a file or variable

    Get-Process | Tee-Object -Variable processes

#### Parameters ####

In [ ]:
Common parameters:

    Debug
    Verbose
    ErrorAction
    ErrorVariable

    InformationAction
    InformationVariable

    WarningAction
    WarningVariable

    OutVariable

    # --- #

    OutBuffer - controls the number of objects that are accumulated in the output buffer before they are sent through the pipeline

    # --- #

    PipelineVariable - assigns the output of the cmdlet to the specified variable, and the original output is still sent through the pipeline
    
    Example:
    Get-Process -PipelineVariable proc | Where-Object {$proc.Id -gt 1000}

In [ ]:
Confirm and ConfirmPreference

    The Confirm switch parameter and the ConfirmPreference variable can be used to decide if a com-
    mand should prompt. The decision to prompt is based on a comparison of ConfirmPreference with
    ConfirmImpact when set by a command author.

    ConfirmPreference has four possible values:
    - High: Prompts when command impact is High (default)
    - Medium: Prompts when command impact is Medium or High
    - Low: Prompts when command impact is Low, Medium, or High
    - None: Never prompts

    ConfirmImpact uses the same four values.

    In Windows PowerShell, the default value for ConfirmImpact is Medium.
    In PowerShell 7, the default value for ConfirmImpact is None. 
    If the command uses SupportsShouldProcess, then the default is Medium

In [ ]:
Confirm

    If the Confirm parameter is explicitly provided, the value of ConfirmPreference within the scope of the
    command is set to Low, which will trigger any confirmation prompts.

    Disable confirmation:
    Remove-Item FileName.txt -Confirm:$false

In [ ]:
ConfirmPreference

    By default, the value of ConfirmPreference is High. This means that prompts will be raised when
    ConfirmImpact for a command is High. The default value for ConfirmPreference may be viewed as
    follows:
    PS> $ConfirmPreference

    In scripts and functions, the ConfirmImpact setting is part of the CmdletBinding attribute

In [ ]:
# WhatIf

    Remove-Item FileName.txt -WhatIf
    # What if: Performing the operation "Remove File" on target "C:\Users\whoami\
    # AppData\Local\Temp\FileName.txt".

In [ ]:
# Parameters splatting

# Parameter from a variable
    $recurse = $false
    Get-ChildItem c:\windows -Recurse:$recurse

# Parameters from a hashtable
    $GetWmiObjectParams = @{
        Class = "Win32_LogicalDisk"
        Filter = "DriveType=3"
        ComputerName = "SERVER2”
        }
    Get-WmiObject @GetWmiObjectParams

# Using backtick
    $taskAction = New-ScheduledTaskAction `
    -Execute pwsh.exe `
    -Argument 'Write-Host "hello world"

In [ ]:
PassThru

    The PassThru parameter is typically used with commands that do not normally generate output and
    is used to force the command to return the object it was working with.

    For example, the Start-Process command does not normally return any output. If PassThru is used,
    it will return the process it created

#### Debugging and logging ####

In [ ]:
# ErrorAction

    $ErrorActionPreference = "Inquire"
    #Continue (Default)
    #Stop

    Get-ChildItem C:\1.txt -ErrorAction Inquire

In [ ]:
# Debug

    Set-PSDebug -Trace 1 #1: Trace script lines as they run. 2: Trace script lines, variable assignments, function calls, and scripts.
    Set-PSDebug -Step #script steping
    Set-PSDebug -Off

In [ ]:
# Trace

    Get-TraceSource | Sort-Object Name | Format-Table -AutoSize

    Trace-Command -Name ParameterBinding, MemberResolution -Expression {Get-MailDomainInfo -DomainName powershellisfun.com} -PSHost

    Trace-Command -Name * -Expression {Get-Process} -PSHost

In [ ]:
#Verbose

    Copy-Item -Path "C:\source\example.txt" -Destination "C:\destination\" -Verbose
    # VERBOSE: Performing the operation "Copy File" on target "Item: C:\source\example.txt Destination: C:\destination\example.txt".

    # Just because a command supports common parameters does not mean it uses them. For
    # example, Get-Process supports the Verbose parameter, yet it does not write any verbose
    # output when Verbose is specified

In [ ]:
# Output Streams commands

    # Write-Output
    # Write-Error
    # Write-Warning
    # Write-Verbose
    # Write-Debug
    # Write-Information

    Write-Error -Message "An error occured" 

In [ ]:
# Start-Transcript

    Start-Transcript -Path "C:\path\to\session.log" -Append

#### Working wih objects ####

In [ ]:
### Get-Member

    $process = Get-Process | Select-Object -First 1
    $process | Get-Member #instance methods
    $process | Get-Member -Static #class methods
    $process.GetType()

    [System.Diagnostics.Process] | Get-Member -Static
    [math] | Get-Member -Static

In [ ]:
# Add-Member, Select-Object

    $person = [PSCustomObject]@{
        Name = "John"
        Age  = 30
    }

# Add the "Email" property
    $person | Add-Member -MemberType NoteProperty -Name "Email" -Value "john@example.com"

# Display Name and Email
    $person | Select-Object Name, Email

In [ ]:
# Add-Member example

    $dir = Get-ChildItem
    $dir | Add-Member -NotePropertyName ComputerName -NotePropertyValue $env:COMPUTERNAME
    $dir | Get-Member ComputerName

In [ ]:
# Select-Object 

    Get-Process -Id $PID | Select-Object *
    Get-Process -Id $PID | Select-Object -Property Name, *Memory*

# ExpandProperty
    Get-Process | Select-Object -First 5 -ExpandProperty Name
    #If ExpandProperty is omitted, the returned object will be a PSCustomObject object with a Name property
    #rather than the simpler array of strings.

# Unique
    1, 1, 1, 3, 5, 2, 2, 4 | Select-Object -Unique
    # Select-Object builds up a collection of unique objects by comparing each property of each object
    # to every unique object that came before it in a pipeline. 
    # When working with strings, the comparison used by Select-Object is case-sensitive

# PropertySet
    Get-Process -Id $PID | Select-Object -Property PSConfiguration

In [ ]:
# Sort-Object

# Sort-Object can be used to make a set of values unique, but unlike Select-Object, comparisons
# between objects are not case-sensitive

    'dog', 'dog', 'cat', 'cat', 'mouse', 'Mouse' | Sort-Object -Unique

In [ ]:
# Where-Object
    Get-Process | Where-Object StartTime -gt (Get-Date 9:00:00)

    Get-Service | Where-Object {
        $_.StartType -eq 'Manual' -and
        $_.Status -eq 'Running'
    }

In [ ]:
# Measure-Object

    dir | Measure-Object -Property Length -Sum #display sum of file sizes

In [ ]:
# Compare-Object

    $a = @{ "one" =  1; "two"= 2 }
    $b = @{ "one" =  1; "three" =  3; "two"= 22 }

    Compare-Object $a $b -Passthru -IncludeEqual -ExcludeDifferent
    Compare-Object @($a.Keys) @($b.Keys) -Passthru -IncludeEqual -ExcludeDifferent

In [ ]:
# Group-Object

Get-Process | Group-Object Company

In [ ]:
# Group-Object example 1

$data = @'
Name|Number|Email|
Bob| 23| bob.bob@test.com|
Jeff|127|jeff.jeff@test.com|
Jeff|129|jeff.jeff@test.com|
Jessica|126|jessica.jessica@test.com|
Jessica|132|jessica.jessica@test.com|
'@ | ConvertFrom-Csv -Delimiter '|'

$data | Group-Object -Property Name, Email |
        Select @{'Name' = 'Name' ; 'Expression' = { $_.Group[0].Name } },
        @{'Name' = 'Number' ; 'Expression' = { $_.Group.Number } },
        @{'Name' = 'Email' ; 'Expression' = { $_.Group[0].Email } }

In [ ]:
# Group-Object example 2

$data = @(
    [pscustomobject]@{Id='1';Service='Service1';Propertyx=1;Price='5'}
    [pscustomobject]@{Id='1';Service='Service2';Propertyx=1;Price='4'}
    [pscustomobject]@{Id='2';Service='Service1';Propertyx=1;Price='17'}
)

$data |Group-Object Id

$data |Group-Object Id |ForEach-Object {
    [pscustomobject]@{
    Id  = $_.Name
    Sum = $_.Group |Measure-Object Price -Sum |ForEach-Object Sum
    #Sum = $_.Group | Select Price
    }
}

#### Variables ####

In [ ]:
# Variables

    $a, $b = 1, 2
    $a = "test1","test2"

    ${env:Path)}

In [ ]:
# Types

    $myVariable = (Get-ChildItem C:\Windows).Name
    $myVariable.GetType() #Array
    $myVariable | Get-Member #String

In [ ]:
# If you had previously statically typed the variable (using something like [int]$a) 
# or if you were in a scope where $a was constrained (like inside a script or function where $a was predefined), 
# then PowerShell could throw an error when you try to reassign it to a different type, such as a hashtable.

    [System.Int32]$a = 10
    $a = "asdf" #error

    [int]$a = 10
    $a = [string]100.1 + [string]9.9
    $intVariable= [double]$intVariable

In [ ]:
# Validation

    [validateRange(1,5)][int]$a = 4
    [validateLength(0,4)][string]$s = 'abed'

In [ ]:
# Commands for variables

    New-Variable -Name pi -Value 3.14 #-Option Constant

    #Get-Variable | Select-Object Name, Description

    Test-Path Variable:pi

    Set-Variable pi -Value 3.1415

    Clear-Variable pi

    Remove-Variable pi -Force

In [ ]:
# User input

    $test = Read-Host "Press Enter"
    $test = @( 'MyDesktop', 'MyLaptop', 'YourDesktop', 'Yourlaptop' ) | Out-GridView -PassThru -Title 'Select your favorite computer'

In [ ]:
Variable scopes

    There are three named scopes:

        Global
        Script
        Local

    Global is the topmost scope; it is the scope the prompt in the console uses and is available to all child
    scopes.

    The Script scope, as the name suggests, is specific to a single script. Script scoped items are available
    to all child scopes (such as functions) within that script. The Script scope is also available in modules,
    making it an ideal place to store variables that should be shared within a module.

    Local is the current scope and is therefore relative. In the console, the Local scope is also the Global
    scope. In a script, the Local scope is the Script scope. Functions and script blocks also have a Local
    scope of their own.

    By default, variables are created in the current scope, the Local scope. However, variables can be
    accessed from the Local scope or any parent scope.

In [ ]:
# Scope modifier

    $Global:variableName = 1
    $Script:variableName2 = 2
    $Local:variableName3 = 3

In [ ]:
# Scope example 1

    function first {
        $first = $name = 'first'
    second
    }
    function second {
        $second = $name = 'second'
    third
    }
    function third {
        "The value of first is $first"
    }

    first

In [ ]:
# Scope example 2

    function first {
        $first = $name = 'first'
    }

    function second {
        $second = $name = 'second'
    }

    function third {
        "The value of first is $first"
    }

    third #The value of first i
    $first = 1
    third #The value of first is 1

In [ ]:
# Private variable

    New-Variable -Name thisValue -Option Private
    $private:thisValue = "Some value"

#### Object types ####

##### Strings #####

In [ ]:
# Interpolation

    'Строка в "одинарных" кавычках'
    #Строка в "одинарных" кавычках

    "Строка в 'двойных' кавычках"
    #Строка в 'двойных' кавычках

    'Строка в ''одинарных кавычках'
    #Строка в 'одинарных кавычках

    "Строка в ""двойных кавычках"
    #Строка в "двойных кавычках

    '$a равно $a'
    #$a равно $a

    "3+2 равно $(3+2)"
    #3+2 равно 5

    "Computer Name: $env:COMPUTERNAME"

    # Wrong - returns an error
    "$ComputerName: Running PS $PSEdition"

    # Right
    $computerName = $env:COMPUTERNAME
    "${ComputerName}: $($ComputerName.Length) characters long"

In [ ]:
# Multiline string

$a=@"
1 Первая строка
$(1+1) Вторая строка
Третья строка"
"@

In [ ]:
# Commands

    Select-String '(Error|Setup)' $env:windir\*.log

In [ ]:
# Replace

    'abababab' -replace 'a', 'c'
    'abababab' -replace 'a'

In [ ]:
# Split

    'a1b2c3d4' -split '[0-9]'

In [ ]:
# Join

    -join ('hello', 'world')
    'a', 'b', 'c', 'd' -join "`t"

In [ ]:
# Regular expressions

    "file.doc" -like "f*.doc"

##### DateTime #####

In [ ]:
# DateTime

#Get date per day
    $dl=Get-Date -Year 2021 -Month 03 -Day 08
    $d2=Get-Date -Year 2021 -Month 02 -Day 23

# Substract dates
    ($dl-$d2).Days #12

# Parsing date
    $Date = "18.12.2024"
    $DateTime = [DateTime]::ParseExact($Date, "dd.MM.yyyy", $null)
    $SQLDate = $Datetime.ToString("yyyy-MM-dd")

# Date formats
    (Get-Date).GetDateTimeFormats()

##### Arrays #####

In [ ]:
# Arrays

    $a = 1..5
    $b = $a

    $a[0] = 3
    $b

In [ ]:
# Array index

    $a[1..2] 
    ""
    $a.indexof(2) #1
    ""
    $a[0, 1 + 3..($a.length-1)]

In [ ]:
# Comparison operators

    $a -gt 3 -lt 5

    $a | Where-Object { $_ -lt 5 -and $_ -gt 3 }

    # The Where method is faster than Where-Object but will raise an error if the variable is not an array.
    $a.where{ $_ -lt 5 -and $_ -gt 3 }

In [ ]:
# Removing elements by index

    $oldArray = 1..100
    $newArray = $oldArray[0..48] + $oldArray[50..99]

In [ ]:
# Nested arrays

    $a = @(1, 2)
    $b = @(1, 2)
    $c =@()
    $c += $a, $b #1 2 1 2

    foreach ($obj in $c) {"object"} # object object
    foreach ($obj in $c) {foreach ($o in $obj) {"object"}} # object object object object

##### Hash tables #####

In [ ]:
# Creating, changing, getting values

    $Employee_Numbers = @{"John Doe" = 112233; "Dave Davis" = 223344; "Justine Smith" = 334455}
    $Employee_Numbers += @{"Scott" = 1;}
    $Employee_Numbers.Add("Richard","2")

    $Employee_Numbers.Scott = 2
    $Employee_Numbers['Scott'] # 2
    $Employee_Numbers["John Doe","Dave Davis"]

    $Employee_Numbers.Remove('Scott')

In [ ]:
#Multiline creation syntax

#Doesn't work if value is an array. 
#; at the end of the lines can be added but are not required

$hash = @{
    'About Arcane Code' = 'http://arcanecode.me'
    ArcaneCode = 'https://arcanecode.com'
    'ArcaneCode RedGate Articles' = 'http://arcanecode.red'
    'ArcaneCode Github Repository' = 'http://arcanerepo.com'}

In [ ]:
# Iteration

    foreach ($h in $hash.GetEnumerator() ) {Write-Host "$($h.Name) : $($h.Value)"}
    foreach ($key in $hash.Keys) { Write-Host "$key : $($hash[$key])" }
    foreach ($h in $hash.Keys) {Write-Host "$h : $($hash.Item($h))"}

In [ ]:
# ContainsKey method

    if (-not $hashtable.ContainsKey('Key3')) {
    $hashtable.Key3 = 'Value3'
    }

In [ ]:
# Comparison

    $a = @{ "one" =  1; "two"= 2 }
    $b = @{ "three" =  3; "two"= 22 }
    Compare-Object @($a.Keys) @($b.Keys) -Passthru -IncludeEqual -ExcludeDifferent

In [ ]:
# OrderedDictionary

    #Hash tables are unordered collections, meaning the order of elements isn't guaranteed
    #An OrderedDictionary preserves the order in which keys were added:

    $ordered = [Ordered]@{
        One = 1
        Two = 2
        Three = 3
    }

##### Type conversion #####

In [ ]:
# -as and -is
    "1" -as [Int32]
    
    'string' -is [String]
    1 -is [Int32]
    [String] -is [Type]
    123 -isnot [String]

#### Data structures ####

In [ ]:
# Multidimensional/jagged arrays
# https://powershellbyexample.dev/post/multidimensional-arrays/

    $array = New-Object 'object[,]' 5,8
    $array[2,5] = 'Hello'
    $array[3,7] = 'World!'
    $array

    ###

    $mdarray1 = @()
    $mdarray1_counter = 0
    $mdarray1 += ,@($mdarray1_counter++, 'Earth',12742)
    $mdarray1 += ,@($mdarray1_counter++, 'Mars',6779)
    $mdarray1[1][1] #Mars

#In PowerShell, the ,@ operator is called the "array splatting" operator. It is used to expand the elements of an array into the encl
#In the given code, the ,@ operator is used to add the array ($mdarray1_counter++, 'Earth',12742) as a single element to the end of the $mdarray1 array. 
#Without the ,@ operator, $mdarray1 += ($mdarray1_counter++, 'Earth',12742) would add 
#the individual elements of the array to $mdarray1 instead of the entire array as a single element.

In [ ]:
### PSObject

<#$data = @'
Name|Number|Email|
Bob| 23| bob.bob@test.com|
Tom|124|tom.tom@test.com|
Jeff|125|jeff.jeff@test.com|
Jeff|127|jeff.jeff@test.com|
Jeff|129|jeff.jeff@test.com|
Jessica|126|jessica.jessica@test.com|
Jessica|132|jessica.jessica@test.com|
'@ | ConvertFrom-Csv -Delimiter '|'

$row = @'
Name|Number|Email|
Nikita|30|a@a.com|
'@ | ConvertFrom-Csv -Delimiter '|'#>

    $data += @([pscustomobject]@{Name="1";Number="2";Email="3"})

    $data | Select-Object -ExcludeProperty Name

    ###

    $data += @(
    [pscustomobject]@{Id='1';Service='Service1';Propertyx=1;Price='5'}
    [pscustomobject]@{Id='1';Service='Service2';Propertyx=1;Price='4'}
    [pscustomobject]@{Id='2';Service='Service1';Propertyx=1;Price='17'})
    $data[1].id = 3

###

#https://www.undocumented-features.com/2019/10/09/creating-an-array-with-header-columns-from-a-string-using-pscustomobject/

[System.Collections.ArrayList]$ArrayWithHeader = @()
$ArrayOfValues = @('a@b.com','b@c.com','c@d.com','d@e.com')
foreach ($obj in $ArrayOfValues) 
{ 
    $val = [pscustomobject]@{'mail'=$obj;'date'=(Get-Date)}
    $ArrayWithHeader.add($val) | Out-Null
}

$ArrayWithHeader

###

$Array0, $Array1 | ForEach-Object {
    [PSCustomObject]@{
        $PSItem[0].Name = $PSItem[0].Value
        $PSItem[1].Name = $PSItem[1].Value
    }
}

In [ ]:
### Datatables

    $tempTable = New-Object System.Data.DataTable

    ###Adding Columns for DataTable###
    $tempTable.columns.Add("col1")
    $tempTable.columns.Add("col2")
    $tempTable.columns.Add("col3")

    #Add a row to DataTable

    $row = $tempTable.NewRow()
    $row[“col1”] = “RandomStringData12”
    $row[“col2”] = “RandomStringData22”
    $row[“col3”] = "RandomStringData32"
    $tempTable.rows.Add($row)

    #If you want to modify a table entry you need to select it using Where-Object or the Where() method.
    $m = $tempTable.where({$_.title -eq 'Justice League'})

    #Then you can modify it. The tricky part to remember is that $m is technically a collection of DataRow objects, even though there is only one.
    $m[0].comments = "DC Comics"

    #Or I could have modified it like this:
    $tempTable.where({$_.title -eq 'Justice League'}).foreach({$_.Comments = "DC Universe"})

#### Flow control ####

##### Comparisons #####

In [ ]:
# -like with string

    $value = 'one'
    $value -like 't*'
    $value -notlike "t*"

In [ ]:
# -like with array

    $value = @('one', 'two', 'three')
    $value -notlike 't*'

In [ ]:
# -match with array

    "one", "two", "three" -match 'e'

In [ ]:
# matches

    'Group one, Group two' -match 'Group (.*), Group (.*)'
    $matches

# The $matches variable is an automatically filled hashtable
# When -match is used against an array, the $matches variable is not set.

In [ ]:
# Left side comparison

    $array = 1, 2, $null, $null
    if ($array -eq $null) { Write-Host 'No values in array' }
    #No values in array

In [ ]:
# Right side comparison

    $array = 1, 2, $null, $null
    if ($null -eq $array) { Write-Host 'Variable not set' }
    #

In [ ]:
# Null-coalescing operator

    $valueA ?? 'Default value'

In [ ]:
# Null-coalescing assignment

    $valueA ??= 'Default value'

In [ ]:
#Null conditional

    $someObject.ToString() #error
    ${someObject}?.ToString()

    $someOtherObject = [PSCustomObject]@{ Value = $null }
    $someOtherObject.Value?.ToString()

In [ ]:
# Chain operators

    Get-Item DoesNotExist -ErrorAction Ignore && Write-Host 'Exists'
    Get-Item DoesNotExist -ErrorAction Ignore || Write-Host 'Does not exist'

##### Iteration #####

In [ ]:
# foreach

    $n = 0
    foreach ($f in dir *.txt) { $n += $f.length }

    $n = 0
    dir *.txt | foreach { $n += length }

In [ ]:
@(
    Get-Date '01/01/2019'
    Get-Date '01/01/2020'
) | ForEach-Object ToString('yyyyMMdd')

In [ ]:
# Parallel

1..10 | ForEach-Object -Parallel {
    Start-Sleep -Seconds 5
    $_
}

##### Loops #####

In [ ]:
# Labeled break statement

    :outerLoop while ($true) {
        while ($true) {
            Write-Host "In inner loop"
            break outerLoop
        }
        Write-Host "This will not be executed"
    }

##### Conditionals #####

In [ ]:
# If

    $result = if ($value) {
        1
    } else {
        2
    }

In [ ]:
# Ternary

    $result = $value ? 1 : 2

    (Get-Process notepad -ErrorAction Ignore) ? 'Running' : 'Not running'

In [ ]:
# Switch example 1

    $msg = "Error, the action failed"

    switch ($msg) {
        { $_ -like "Error*" } { "Action error" }
        { $_ -like "Warning*" } { "Action warning" }
        { $_ -like "Successful*" } { "Action succesfull" }
    }

In [ ]:
# Switch example 2

    $a = 3

    switch ($a) {
        1 {"Один"}
        2 {"Два"}
        3 {"Три"; break}
        4 {"Четыре"}
        3 {"Еще раз три"}
    }
    #Три

In [ ]:
# Switch example 3

    switch (3) {
        1 {"1"}
        2 {"2"}
        default {"none"}
    }

In [ ]:
# Switch example 4

    switch -wildcard ('абв') {
        'а*' {"$_ начинается с а"}
        '*в' {"$_ оканчивается на в"}
    }

In [ ]:
# Switch example 5

    switch (10) {
        {$_ -gt 5} {"$_ >5"}
        {$_ -lt 20} {"$_ <20"}
        10 {"$_ =10"}
    }

In [ ]:
# Switch example 6

    switch -wildcard (dir $env:SystemRoot) {
        *.txt {$txt++}
        *.log {$log++} 
    }

In [ ]:
# Switch example 7

    switch -wildcard -file $env:SystemRoot\KB946627.log {
        *Source:* {$_}
        *Destination:* {$_}
    }

#### Functions ####

In [ ]:
# Function example 1

    function MyFunc { Get-Command -Noun Process }

In [ ]:
# Function example 2 - parameters

    function Set-ArchiveFilePath{
        [CmdletBinding()]
        [OutputType([string])]
        param(
            [Parameter(Mandatory = $true)]
            [string]$ZipPath,
            [Parameter(Mandatory = $true)]
            [string]$ZipPrefix,
            [Parameter(Mandatory = $true)]
            [datetime]$Date
        )

        Write-Host "this is a function code"

    }

In [ ]:
# Function example 3

    function split($items, $splitter) {
        ForEach ($item in $items) {
            $item -split "\$splitter" 
        }
    }

    split 'asd\f' \ #asd, f

In [ ]:
# Function example 4

    function Output-SalesTax ( $Price, [int]$Tax ) { $Price / $Tax }
    
    Output-SalesTax -price 1000 -tax 38
    Output-SalesTax -tax 38 -price 1000 
    Output-SalesTax 1000 38

In [ ]:
# Function example 5

    function Get-UserInfo {
        param (
            [string]$Name,
            [int]$Age,
            [string]$Email
        )

        Write-Host "Name: $Name"
        Write-Host "Age: $Age"
        Write-Host "Email: $Email"
    }

    Get-UserInfo -Name "John Doe" -Age 30 -Email "john.doe@example.com"

In [ ]:
# Function example 6

    function MyFunc ([switch] $recurse) {
        if ($recurse) { "#"}
        else { "Regular function" } }
    MyFunc -recurse #Recursive function

    #Write-host выводит строку на экран, а не в выходной поток, поэтому она не попадет в значение переменной

In [ ]:
Pipeline to a function

    Если в функцию могут поступать какие-либо параметры по конвейеру, то в ней
    обязательно нужно Определить раздел Process.
    Для эффективного
    использования конвейеров возвращать значения в расширенных функциях следует
    в разделе Process (напомним, что он выполняется каждый раз при получении по
    конвейеру нового объекта), а не в разделах Begin или End
    В расширенных функциях всегда нужно указывать атрибут [CmdietBindingO ],
    в этом случае функция будет вести себя как командлет, в том числе будет поддерживать
    флаги -verbose (вывод подробной информации о выполняемых командах)
    и -Debug (вывод отладочной информации).

In [ ]:
# Pipeline to a function

    function ProcessFiles {
        param (
            [string]$Prefix
        )
        
        process {
            # Example action: Write the name of each file with the prefix
            Write-Host "$($Prefix): $($_.Name)"
        }
    }
    
    Get-ChildItem | ProcessFiles -Prefix "File"

In [ ]:
# Filter function to be used in a pipeline

    filter FileSizeBelow($size) { if ($_.length -le $size) { $_ } }
    gci C:\Windows | FileSizeBelow 200kb

In [ ]:
When returning data from a function, it is best to save it to a variable in the function
and have just that variable on the last line of the function

In [ ]:
# Lambda / Script-block - 1

    $alert = { "Hello World" }
    & $alert

In [ ]:
Lambda / Script-block - 2

Each time the scriptblock is run; it will dynamically read the current value of the variable.
When a scriptblock is run using the “&” (call) operator, updates to a variable are not reflected in the parent scope.
When a scriptblock is run using the “.” (dot) operator, updates to a variable apply to the current scope.
Begin {}, Process {} and End {} blocks can be added to a scriptblock, just like a function

In [ ]:
# Lambda / Script-block - 3

    $add = { param($a, $b) $a + $b }
    $result = &$add 5 3
    Write-Host "Result: $result"

    $add = { 3 + 5 }
    $result = &$add
    Write-Host "Result: $result"

#### Meta ####

In [ ]:
# Alias

    Set-Alias list Get-Location
    Set-Alias np c:\windows\notepad.exe
    Get-Alias

In [ ]:
# Methods

    $methodName = "ToUpper"
    $object = "hello"
    $object.GetType().GetMethod($methodName).Invoke($object, $null)

    $method = [scriptblock]::Create('"hello".ToUpper()')
    & $method

#### Other ####

In [ ]:
### Working with PSDrives and Providers

    Get-PSDrive
    Get-PSProvider

    Get-ChildItem #dir, ls
    Get-ChildItem registry::

    New-PSDrive -Name docs -PSProvider FileSystem -Root C:\Users\

    New-Item

    Remove-Item

In [ ]:
# Working with files

    Get-Content file.txt | 
        Where-Object { $_ -eq 'first' } | 
    Set-Content file.txt
#Set-Content: The process cannot access the file 'C:\temp\file.txt' because it 
#is being used by another process.

#This can be avoided by ensuring that all content is read before the pipeline starts. This can be achieved 
#by enclosing the fi rst command in the pipeline in brackets: 

    (Get-Content file.txt) | 
        Where-Object { $_ -eq 'first' } | 
    Set-Content file.txt

#Raw parameter reads the entire file as a single string
#The Get-Content command is slow when reading from large fi les without the Raw parameter. 

In [ ]:
# Self-Update

    Copy-Item -Path $ScriptUpdateLocation -Destination $mypath -Force
    Get-Content $ScriptUpdateLocation -Raw | Set-Content $PSCommandPath
    #$scriptContent | Out-File -FilePath "$PSScriptRoot\Check_SQL_Logins_Fixed.ps1" -Encoding utf8
    Invoke-Expression -Command $scriptContent

In [ ]:
# Bonus

    ( 392, 330, 349, 247, 330, 349, 392, 330 ) | ForEach { [Console]::Beep( $_, 500 ) } # use 32767 for pauses